In [1]:
%reload_ext autoreload
%autoreload 2

In [5]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex

data: DiffResult = load_pkl("src/summary")


col_names = {
    IssueEnum.OLD: r"\multicolumn{1}{c}{\ No.}",
    IssueEnum.ADD: r"\multicolumn{1}{c}{$+$\tiny\%}",
    IssueEnum.REMOVE: r"\multicolumn{1}{c}{$-$\tiny\%}",
    IssueEnum.CHANGE: r"\multicolumn{1}{c|}{$\Delta$\tiny\%}",
}

group_names = {
    VersionGroup.LTS: "LTS (2 yr)",
    VersionGroup.REGULAR: "Regular Releases (6 mo)",
    # VersionGroup.REV: "Revisions for 5.4",
}


results = {}

for (group, pair), diffs in data.items():
    if group not in group_names:
        continue

    result = {}
    old = 0
    for (kind, key), s in diffs.items():
        if kind == DepKind.SYSCALL:
            continue

        if key not in col_names.keys():
            continue

        display_key = col_names[key]

        if key == "Old":
            old = s
            if kind == DepKind.FUNC:
                s = f"{s / 1000:.0f}k"
            elif kind == DepKind.STRUCT:
                s = f"{s / 1000:.1f}k"
        else:
            if s == 0:
                s = GRAY_DASH
            else:
                s = f"{s / old * 100:.0f}"

        s = r"\multirow{2}{*}{" + str(s) + "}"
        result[(kind, display_key)] = s

    v1 = group.version_to_str(pair.v1)
    v2 = group.version_to_str(pair.v2)

    group_name = group_names[group]
    results[(group_name, v1)] = result
    if pair.v2 == list(group)[-1]:
        results[(group_name, v2)] = {k: "" for k in result.keys()}


df = pd.DataFrame(results).T

latex = df.to_latex(multicolumn_format="c|", column_format="cc|rrrr|rrrr|rrrr|rrrr")
save_latex(latex, "src")


# df

[        utils.py:19 ] INFO: Loding src/summary from /Users/szhong/Downloads/bpf-study/output/src/summary.pkl
[        utils.py:30 ] INFO: Saved src to /Users/szhong/Downloads/bpf-study/paper/tabs/src.tex


In [6]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex, use_midrule

data: DiffResult = load_pkl("src/summary")


def multicolumn(s, n=2, format="c"):
    return r"\multicolumn{2}{ " + format + r"}{" + s + "}"


results = {}
for (group, pair), diffs in data.items():
    if group != VersionGroup.LTS:
        continue

    result = {}
    change = 0
    for (kind, key), v in diffs.items():
        if v == 0:
            continue
        if kind == DepKind.SYSCALL:
            continue
        if key in (IssueEnum.ADD, IssueEnum.REMOVE, IssueEnum.OLD, IssueEnum.NEW):
            continue
        if key == IssueEnum.CHANGE:
            change = v
            if v > 1000:
                s = f"{v / 1000:.1f}k"
            else:
                s = str(v)
            if kind == DepKind.TRACEPOINT:
                kind = "Tracept"
            result[(kind, "No. changed")] = s
            continue

        if kind == DepKind.TRACEPOINT:
            if key not in (IssueEnum.TRACE_EVENT_CHANGE, IssueEnum.TRACE_FUNC_CHANGE):
                continue
            kind = "Tracept"

        result[(kind, key)] = f"{v / change * 100:.0f}" + r"\%"

    v1 = group.version_to_str(pair.v1)
    v2 = group.version_to_str(pair.v2)

    def header(v):
        return ("Linux Kernel Version", v)

    if pair.v1 == list(group)[0]:
        results[header(v1)] = {k: "" for k in result.keys()}
    results[header(v2)] = result

df = pd.DataFrame(results)
df = df.fillna(GRAY_DASH)
df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: x + " &")
latex = df.to_latex(multicolumn_format="c", column_format="cl|" + "r" * 7)
latex = latex.replace(
    r"\multicolumn{5}{c}{Linux Kernel Version}",
    r"\multicolumn{6}{c}{Linux Kernel Version}",
)
latex = latex.replace(
    "4.4 & 4.15 & 5.4 & 5.15 & 6.8",
    r"\multicolumn{6}{c}{4.4 \quad 4.15 \quad 5.4 \quad 5.15 \quad 6.8}",
)
latex = use_midrule(latex)
save_latex(latex, "breakdown")
df

[        utils.py:19 ] INFO: Loding src/summary from /Users/szhong/Downloads/bpf-study/output/src/summary.pkl
[        utils.py:30 ] INFO: Saved breakdown to /Users/szhong/Downloads/bpf-study/paper/tabs/breakdown.tex


Linux Kernel Version                                          
                                              4.4  4.15                   5.4  5.15     6.8
Function No. changed                               2.0k                  1.8k  2.2k  3.0k &
         Param added                               57\%                  53\%  60\%  51\% &
         Param removed                             41\%                  36\%  44\%  48\% &
         Param reordered                           19\%                  19\%  25\%  19\% &
         Param type changed                        26\%                  23\%  26\%  25\% &
         Return type changed                       15\%                  21\%  13\%  17\% &
Struct   No. changed                               1.4k                  1.3k  1.6k  1.7k &
         Field added                               59\%                  62\%  63\%  65\% &
         Field removed                             34\%                  34\%  34\%  37\% &
         Field type changed                        30\%                  29\%  27\%  30\% &
Tracept  No. changed                                 39                    54   120   115 &
         Event changed                             95\%                  81\%  85\%  92\% &
         Func changed                              46\%                  54\%  32\%  51\% &
LSM Hook No. changed                                 19                    17    27    27 &
         Param added                               16\%                  24\%  74\%  63\% &
         Param removed                              5\%                  35\%  56\%  59\% &
         Param reordered                            5\%  \color{lightgray}{-}  15\%   4\% &
         Param type changed                        89\%                  65\%  26\%  41\% &